In [10]:
import qiskit
from qiskit.quantum_info import state_fidelity
import numpy as np
from numpy import linalg as LA
import qib
import matplotlib.pyplot as plt
import scipy

I2 = np.eye(2)
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

# Random unitary generator
def random_unitary(n):
    A = np.random.randn(n, n) + 1j * np.random.randn(n, n)
    Q, _ = np.linalg.qr(A)
    return Q

L = 4
# construct Hamiltonian
latt = qib.lattice.IntegerLattice((L,), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
#hamil = qib.IsingHamiltonian(field, 1, 0, 0).as_matrix().toarray()
hamil = qib.HeisenbergHamiltonian(field, (1,1,1), (0,0,0)).as_matrix().toarray()
perms = [[i for i in range(L)], [i for i in range(1, L)]+[0], [i for i in range(L)]]
#perms = [[i for i in range(L)]]

t = 0.1
U = scipy.linalg.expm(-1j*t*hamil)
U_back = scipy.linalg.expm(1j*t*hamil)
cU = U_back

In [ ]:
import sys
sys.path.append("../ccU")
from utils import otimes, applyG
from ansatz import ansatz, ansatz_grad_vector
from hessian import ansatz_hessian_matrix
from optimize import optimize_circuit


Glists = []
f_iters = []
err_iters = []
for _ in range(50):
    Glist_start = [random_unitary(4) for _ in range(len(2*perms))]
    #Glist_start = Glist_perms3_best_L4
    Glist, f_iter, err_iter = optimize_circuit(L, U, cU, Glist_start, perms, niter=100)
    print("Best f: ", f_iter[-1])
    print("Best err: ", err_iter[-1])
    Glists.append(Glist)
    f_iters.append(f_iter)
    err_iters.append(err_iter)

In [13]:
import h5py

with h5py.File(f"../results_data/Heisenberg1d_L{L}_t{t}_layers{len(perms)}_err{round(err_iters[1][-1], 2)}.hdf5", "w") as f:
    f.create_dataset("Glist", data=Glists[1])
    f.create_dataset("f_iter", data=f_iters[1])

In [8]:
from optimize import err

L = 4
latt = qib.lattice.IntegerLattice((L,), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
hamil = qib.HeisenbergHamiltonian(field, (1,1,1), (0,0,0)).as_matrix().toarray()
perms = [[i for i in range(L)], [i for i in range(1, L)]+[0], [i for i in range(L)]]
U = scipy.linalg.expm(-1j*t*hamil)
U_back = scipy.linalg.expm(1j*t*hamil)
cU = U_back

err_opt = 0.0

Glist_perms3_best_L4 = None
with h5py.File(f"../results_data/Heisenberg1d_L{L}_t{t}_layers{len(perms)}_err{err_opt}.hdf5", "r") as f:
    Glist_perms3_best_L4 = f["Glist"][:]
    f_iter_L4_best = f["f_iter"][:]
print("Reusing Best L=4 gates for L=4, Frobenius Norm: ", np.linalg.norm(ansatz(Glist_perms3_best_L4, U, L, perms) - cU, ord=2))
print("Cost function: ", err(Glist_perms3_best_L4, U, L, perms, cU))

Glist_perms3_best_L2 = None
with h5py.File(f"../results_data/Heisenberg1d_L2_t1_layers3.hdf5", "r") as f:
    Glist_perms3_best_L2 = f["Glist"][:]
print("Reusing Best L=2 gates for L=4, Frobenius Norm: ", np.linalg.norm(ansatz(Glist_perms3_best_L2, U, L, perms) - cU, ord=2))
print("Cost function: ", err(Glist_perms3_best_L2, U, L, perms, cU))

Reusing Best L=4 gates for L=4, Frobenius Norm:  1.7817328573034614e-09
Cost function:  -16.000000000000018
Reusing Best L=2 gates for L=4, Frobenius Norm:  1.9999509729452152
Cost function:  -3.4358484041193478


In [9]:
L = 6
latt = qib.lattice.IntegerLattice((L,), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
hamil = qib.HeisenbergHamiltonian(field, (1,1,1), (0,0,0)).as_matrix().toarray()
perms = [[i for i in range(L)], [i for i in range(1, L)]+[0], [i for i in range(L)]]
U = scipy.linalg.expm(-1j*t*hamil)
U_back = scipy.linalg.expm(1j*t*hamil)
cU = U_back

print("Reusing Perfect L=4 gates for L=6, Frobenius Norm: ", np.linalg.norm(ansatz(Glist_perms3_best_L4, U, L, perms) - cU, ord=2))
print("Cost function: ", err(Glist_perms3_best_L4, U, L, perms, cU))

Reusing Perfect L=4 gates for L=6, Frobenius Norm:  1.9569208103394087
Cost function:  -11.809462526790336
